In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns 
import os

# =========================================================
# ⚙️ KONFIGURASI
# =========================================================
FOLDER_DATA = "/open_meteo_climate"
NAMA_FILE = "kebumen_75tahun_lengkap.csv" 
PATH_FILE = os.path.join(FOLDER_DATA, NAMA_FILE)

In [7]:
# =========================================================
# 1. LOAD DATA
# =========================================================
def load_data(filepath):
    if not os.path.exists(filepath):
        print(f"❌ File tidak ditemukan: {filepath}")
        return None
    try:
        df = pd.read_csv(filepath, index_col='date')
        df.index = pd.to_datetime(df.index, utc=True, errors='coerce')
        df.index = df.index.tz_convert('Asia/Jakarta')
        df.index = df.index.tz_localize(None)
        df = df.sort_index()
        return df
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# =========================================================
# 2. FUNGSI ANALISIS DETAIL (DISIMPLIFIKASI)
# =========================================================
def analisis_detail(df, target_waktu):
    print(f"\n🔎 ANALISIS DETAIL PERIODE: {target_waktu}")
    print("="*50)
    
    try:
        subset = df.loc[target_waktu]
        if subset.empty:
            print("⚠️ Data tidak ditemukan.")
            return

        # --- A. ANALISIS SUHU ---
        print("🌡️ STATISTIK SUHU")
        
        # Hitung Rata-rata
        avg_temp = subset['temperature'].mean()
        
        # Cari Max & Min
        max_temp = subset['temperature'].max()
        max_temp_date = subset['temperature'].idxmax()
        min_temp = subset['temperature'].min()
        min_temp_date = subset['temperature'].idxmin()
        
        print(f"      Rata-rata Suhu: {avg_temp:.2f}°C") # <--- TAMBAHAN
        print(f"   🔥 Paling Panas  : {max_temp:.2f}°C (pada {max_temp_date})")
        print(f"   ❄️ Paling Dingin : {min_temp:.2f}°C (pada {min_temp_date})")
        print("-" * 30)

        # --- B. ANALISIS HUJAN (KATEGORI BARU) ---
        print("🌧️ STATISTIK HUJAN")
        
        # Agregasi Harian
        harian = subset['rain_mm'].resample('D').sum()
        
        # 1. Total & Max
        total_hujan = harian.sum()
        max_hujan_sehari = harian.max()
        tgl_max_hujan = harian.idxmax().strftime('%Y-%m-%d')
        
        print(f"🌧️Total Curah Hujan : {total_hujan:.2f} mm")
        print(f"🌧️Hujan Terlebat   : {max_hujan_sehari:.2f} mm (Tgl {tgl_max_hujan})")
        
        # 2. Kategori Sederhana (Basah vs Kering)
        # Hari Basah: > 10 mm
        hari_basah = harian[harian > 10]
        jml_hari_basah = len(hari_basah)
        
        # Hari Kering: <= 10 mm (Termasuk 0 mm dan gerimis)
        hari_kering = harian[harian <= 10]
        jml_hari_kering = len(hari_kering)
        
        print(f"☔ Hari Basah (>10mm)  : {jml_hari_basah} hari")
        print(f"☀️ Hari Kering (<=10mm): {jml_hari_kering} hari")
        
        print("="*50)

    except KeyError:
        print(f"❌ Format waktu salah. Gunakan 'YYYY' atau 'YYYY-MM'.")
    except Exception as e:
        print(f"⚠️ Error analisis: {e}")

# =========================================================
# 3. FUNGSI PLOTTING
# =========================================================
def plot_periode_tertentu(df, target_waktu):
    print(f"\n🎨 Menggambar grafik untuk periode: {target_waktu}...")
    try:
        subset = df.loc[target_waktu]
        if subset.empty: return

        fig, ax1 = plt.subplots(figsize=(12, 6))
        
        # Plot Suhu
        ax1.plot(subset.index, subset['temperature'], 
                 color='darkorange', marker='', linestyle='-', linewidth=1.5, label='Suhu (°C)')
        ax1.set_ylabel('Suhu (°C)', color='darkorange', fontweight='bold')
        ax1.tick_params(axis='y', labelcolor='darkorange')
        ax1.grid(True, linestyle='--', alpha=0.5)
        
        # Plot Hujan
        ax2 = ax1.twinx()
        width = 0.04 if len(str(target_waktu)) > 7 else 0.5 
        ax2.bar(subset.index, subset['rain_mm'], 
                color='dodgerblue', alpha=0.4, width=width, label='Hujan (mm)')
        ax2.set_ylabel('Hujan (mm)', color='dodgerblue', fontweight='bold')
        ax2.tick_params(axis='y', labelcolor='dodgerblue')
        
        # Format Sumbu X
        if len(str(target_waktu)) > 7:
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
            ax1.set_xlabel('Jam (WIB)')
            judul = f"Cuaca Detail Tanggal: {target_waktu}"
        elif len(str(target_waktu)) == 7:
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d'))
            ax1.set_xlabel('Tanggal')
            judul = f"Tren Cuaca Bulan: {target_waktu}"
        else:
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
            ax1.set_xlabel('Bulan')
            judul = f"Tren Cuaca Tahun: {target_waktu}"

        plt.title(judul, fontweight='bold', fontsize=14)
        
        lines_1, labels_1 = ax1.get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"⚠️ Terjadi error saat plotting: {e}")

In [8]:
# =========================================================
# EKSEKUSI UTAMA
# =========================================================
if __name__ == "__main__":
    df_cuaca = load_data(PATH_FILE)
    
    if df_cuaca is not None:
        # Contoh Analisis Bulan Oktober 2022 (Bulan Basah)
        TARGET = "2025-11"
        
        # 1. Tampilkan Angka Statistik
        analisis_detail(df_cuaca, TARGET)
        
        # 2. Tampilkan Grafik
        plot_periode_tertentu(df_cuaca, TARGET)

❌ File tidak ditemukan: /open_meteo_climate\kebumen_75tahun_lengkap.csv


In [5]:
df_cuaca.tail(10)

AttributeError: 'NoneType' object has no attribute 'tail'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=672c3bf7-b636-46e8-b7c0-4f6feec29313' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>